In [2]:
import os
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et


In [3]:
root =  os.path.abspath(os.path.curdir)
work_path = os.path.join (root, 'Dataset', 'AFO_before')
images_path = os.path.join(work_path, 'AFO_images')
annot_path = os.path.join(work_path, 'AFO_labels_XML')
convert_annot_path = os.path.join(work_path, 'AFO_labels_txt')
if os.path.exists(work_path):
    print('work_path is OK!')
else:
    print('work_path not Exists!')        
if os.path.exists(images_path):
    print('images_path is OK!')
else:
    print('images_path not Exists!')        
if os.path.exists(annot_path):
    print('annot_path is OK!')
else:
    print('annot_path not Exists!')        
if os.path.exists(convert_annot_path):
    print('convert_annot_path is OK!')    
else:
    print('convert_annot_path not Exists!')        


work_path is OK!
images_path is OK!
annot_path is OK!
convert_annot_path is OK!


In [4]:
def get_annot_pandas(filename):
    tree=et.parse(filename)
    root=tree.getroot()
    class_list = []
    fname = os.path.split(filename)[-1]
    # print(fname)
    df_annot = pd.DataFrame(columns=['filename', 'object_class','x', 'y','w', 'h'])
    for element in root:
        if element.tag =='size':
            for item in element:
                if item.tag == 'width':
                    w = int(item.text)
                if item.tag == 'height':
                    h = int(item.text)
        if element.tag =='object':
            for item in element:
                if item.tag == 'name':
                    class_name = item.text.replace('/','_')
                    if not(class_name in class_list):
                        class_list.append(class_name)
                if item.tag == 'bndbox':
                    for ancor in item:
                        if ancor.tag =='xmin':
                            l = int (ancor.text)
                        if ancor.tag =='ymin':
                            t = int (ancor.text)
                        if ancor.tag =='xmax':
                            r = int (ancor.text)
                        if ancor.tag =='ymax':
                            b = int (ancor.text)
                    object_w = (r-l)/w
                    object_h = (b-t)/h
                    object_x = (l+(r-l)/2)/w
                    object_y = (t+(b-t)/2)/h
            # print(class_name, object_x,object_y,object_w,object_h)    
            df_annot.loc[df_annot.size]=[fname, class_name, object_x,object_y,object_w,object_h]                        
    return df_annot

In [5]:
img_extension = ['.jpg']
ann_extension = ['.xml']
ann_txt_exten = ['.txt']
filelist = []
df_summary = None
for root, directories, filenames in os.walk(images_path, topdown=True):
    for filename in filenames[:]:
        _, ext = os.path.splitext(filename)
        if ext in img_extension:
            filelist.append(os.path.join(root, filename)) 
            name, ext = os.path.splitext(filename)
            filename = f'{name}{ann_extension[0]}'
            fname = os.path.join(annot_path, filename)
            if os.path.isfile(fname):
                df = get_annot_pandas(fname)
                if df_summary is None:
                    df_summary = df
                else:
                    df_summary = pd.concat((df_summary, df))
                    
            
display(df_summary.head(10))            
print(f'Найдено {len(filelist)} файлов')


,filename,object_class,x,y,w,h
0,av_1381.xml,human,0.341146,0.503704,0.013542,0.024074
6,av_1381.xml,human_west,0.377995,0.355556,0.020052,0.019444
12,av_1381.xml,human_west,0.416146,0.397917,0.015104,0.029167
18,av_1381.xml,human,0.474349,0.281944,0.020573,0.027778
24,av_1381.xml,human_west,0.456250,0.471759,0.017188,0.042593
30,av_1381.xml,human,0.493620,0.504630,0.020573,0.025926
36,av_1381.xml,human,0.504818,0.556481,0.032031,0.046296
42,av_1381.xml,human,0.444271,0.667130,0.016146,0.034259
48,av_1381.xml,human_west,0.548438,0.528704,0.021875,0.025926
54,av_1381.xml,human_west,0.584115,0.501620,0.024479,0.062500


Найдено 1704 файлов


In [6]:
df_summary['object_class'].value_counts(dropna=False)

human              8165
human_west         5852
boat               3517
human_boat_west    2315
human_boat          962
bouy                605
Name: object_class, dtype: int64

In [8]:
convert_dict = {'human_boat': 2 , 'human_boat_west': 3 , 'boat': 4,'human': 0, 'human_west': 1, 'bouy':5 }

In [9]:
df_summary = df_summary.replace({"object_class": convert_dict})

In [10]:
df_summary['object_class'].value_counts(dropna=False)

0    8165
1    5852
4    3517
3    2315
2     962
5     605
Name: object_class, dtype: int64

In [12]:
for filename in df_summary['filename'].values[:]:
    name, ext = os.path.splitext(filename)
    fname = f'{name}{ann_txt_exten[0]}'
    df_one = df_summary.loc[df_summary['filename'] ==filename]
    filepath = os.path.join(convert_annot_path, fname)
    df_one[['object_class', 'x','y','w','h']].to_csv(filepath,header=False, index=False, sep=' ')
    # display(df_one)